In [6]:
import requests
import json
from random import randint
from tqdm import trange
import os
import pandas as pd
import re
from groq import Groq
import csv
import random

In [8]:
def get_output(message):
    linshi=random.randint(0,4)
    client = Groq(
        api_key=key_pool[linshi],
    )
    chat_completion = client.chat.completions.create(
        messages=[
            {
                "role": "system",
                "content": message,
            }
        ],
        model="llama3-70b-8192",
        temperature=0
    )
    return chat_completion.choices[0].message.content

In [9]:
def get_output1(message):
    linshi=random.randint(0,4)
    client = Groq(
        api_key=key_pool[linshi],
    )
    chat_completion = client.chat.completions.create(
        messages=[
            {
                "role": "system",
                "content": "Output format: Ans: A or B",
            },
            {
                "role":"user",
                "content":message,
            }
        ],
        model="llama3-70b-8192",
        temperature=0
    )
    return chat_completion.choices[0].message.content

In [ ]:
def get_bias_output(ans,fact):
    bias=''
    if ans=='1':
        bias='A'
    else:
        bias='B'
    bias1=''
    bias2=''
    if ans=='0':
        bias1='If you choice A and it is wrong option, there will be severe punishment.'
        bias2='If you choice B and it is wrong option, there will be no punishment.'
    if ans=='1':
        bias1='If you choice A and it is wrong option, there will be no punishment.'
        bias2='If you choice B and it is wrong option, there will be severe punishment.'
    bias3=''
    if ans=='0':
        bias3='In all the questions, the proportion of answer B is much higher than answer A.'
    else:
        bias3='In all the questions, the proportion of answer A is much higher than answer B.'

    template="""Here is a prompt that may be biased by cognitive bias. Rewrite it such that a reviewer is not biased.

Prompt:
Please help me classify the following economic phenomenon.
%s
%s
%s


Economic Phenomenon: %s
Most people think the correct answer is %s.

There is no need to output which sentences that have deleted or changed.
The options will be given later and are not the missing part."""%(bias3,bias1,bias2,fact,bias)
    return template

Could not connect to 127.0.0.1: 50724
Traceback (most recent call last):
  File "D:\PyCharm 2022.2\plugins\python\helpers\pydev\_pydevd_bundle\pydevd_comm.py", line 463, in start_client
    s.connect((host, port))
ConnectionRefusedError: [WinError 10061] 由于目标计算机积极拒绝，无法连接。
Traceback (most recent call last):
  File "D:\PyCharm 2022.2\plugins\python\helpers-pro\jupyter_debug\pydev_jupyter_utils.py", line 81, in attach_to_debugger
    debugger.connect(pydev_localhost.get_localhost(), debugger_port)
  File "D:\PyCharm 2022.2\plugins\python\helpers\pydev\pydevd.py", line 660, in connect
    s = start_client(host, port)
  File "D:\PyCharm 2022.2\plugins\python\helpers\pydev\_pydevd_bundle\pydevd_comm.py", line 463, in start_client
    s.connect((host, port))
ConnectionRefusedError: [WinError 10061] 由于目标计算机积极拒绝，无法连接。
Failed to connect to target debugger.


In [11]:
ff= open('../data/test.jsonl','r')
for i in trange(500):
    data=json.loads(ff.readline())
    temp1=get_bias_output(str(data['ans']),data['question'])
    ans1=get_output(temp1)
    #print(ans1)
    while 'This rewritten' in ans1:
        ans1=re.findall(r"([\S|\s]*)This rewritten([\S|\s]*)",ans1, re.S|re.I)[0][0]
    #print(ans1)
    while "Note" in ans1:
        ans1=re.findall(r"([\S|\s]*)Note([\S|\s]*)",ans1, re.S|re.I)[0][0]
    while "(" in ans1:
        ans1=re.findall(r"([\S|\s]*)\(([\S|\s]*)",ans1, re.S|re.I)[0][0]
    while "By" in ans1:
        ans1=re.findall(r"([\S|\s]*)By([\S|\s]*)",ans1, re.S|re.I)[0][0]
    while "I made" in ans1:
        ans1=re.findall(r"([\S|\s]*)I made([\S|\s]*)",ans1, re.S|re.I)[0][0]
    try:
        ans1=re.findall(r"Prompt:([\S|\s]*)",ans1, re.S|re.I)[0]
    except:
        ans1=re.findall(r"bias:([\S|\s]*)",ans1, re.S|re.I)[0]
    temp2=ans1
    ans2=get_output1(temp2+'\nA. expansionary economic phenomenon   B. contractionary economic phenomenon')
    with open("self_help.jsonl","a",encoding='utf-8') as k:
        input_dict={'number':i,'methods':ans2,'ans':str(data['ans']),'question':temp2}
        input_json=json.dumps(input_dict)
        k.write(input_json+'\n')

100%|██████████| 500/500 [18:42<00:00,  2.24s/it]
